In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import genemunge

# Searching the gene ontology for relevant genes

In [ ]:
# set up an object to search the gene ontology
searcher = genemunge.search.Searcher()

# get all of the GO identifiers associated with the word 'immune'
# set exact = False to walk through the ontology and grab all child terms
immune_identifiers = searcher.keyword_search(['immune'], exact=False)

# get all of the genes assigned to the immune_identifiers
immune_genes = searcher.get_genes(immune_identifiers)

# get a list of housekeeping genes
housekeeping = searcher.get_housekeeping_genes()

# keep all of the immune related genes that are not housekeeping genes
variable_immune_genes = list(set(immune_genes) - set(housekeeping))

print('Identified {} variable immune related genes'.format(len(variable_immune_genes)))

# Converting between gene identifier types

In [ ]:
# set up an object to convert from ensembl to symbol
ensembl_to_symbol = genemunge.convert.IDConverter('ensembl_gene_id', 'symbol')

# convert the immune identifiers to gene symbols
variable_immune_symbols = ensembl_to_symbol.convert_list(variable_immune_genes)

# Obtaining statistics about gene expression

In [ ]:
# set up an object to describe genes
describer = genemunge.describe.Describer('symbol')

In [ ]:
# find the absolute and relative expression levels of each gene of interest
expression_data = pd.DataFrame(index=variable_immune_symbols,
                               columns=['expression', 'log ratio'])
for gene in variable_immune_symbols:
    # get the expression levels in healthy tissue (in TPM units)
    try:
        stats = describer.get_tissue_expression(gene)
        stomach_med = stats['median'].loc['Stomach']
        small_intestine_med = stats['median'].loc['Small Intestine']
    # some genes are not in GTEx - set expression to 0
    except KeyError:
        stomach_med = 0.0
        small_intestine_med = 0.0
    # control the log with a small pseudocount
    pseudocount = 1.0
    expr_ratio = (pseudocount+small_intestine_med) / (pseudocount+stomach_med)
    expression_data.loc[gene] = [small_intestine_med, np.log10(expr_ratio)]

In [ ]:
# plot the gene expression fraction
fig, ax = plt.subplots(figsize=(12, 8))
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

ax.scatter(expression_data['expression'], expression_data['log ratio'])
ax.set_xlabel('Small Intestine expression [TPM]', fontsize=20)
ax.set_ylabel('log10 ratio (Small Intestine / Stomach)', fontsize=20)
ax.set_xscale('log')
ax.set_xlim([0.001, 1e5])
ax.set_ylim([-3, 4])
plt.savefig('small_intestine_example.png', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
target_genes = expression_data[expression_data['log ratio'] > 1]
target_genes = target_genes.sort_values(by=['expression'], ascending=False)
target_genes

# Getting information about a specific gene

In [ ]:
# set up an object to describe genes
describer = genemunge.describe.Describer('symbol')

# get some basic information about one of the immune genes
print(json.dumps(describer.get_gene_info(target_genes.index[0]), indent=2))

# make a plot of the expression of one of the immune genes across tissues from GTEx
describer.plot_tissue_expression(target_genes.index[0], sortby='median',
                                 filename='gene_expr_example.png')